# Parsing and Processing Lookup Responses

The purpose of this notebook is to analyze the offers scraped for each provider and join each offer scraped with census information about eahc address the offer was scraped for.

In [1]:
import os
import glob
import gzip
import json
from datetime import datetime

import multiprocess
import numpy as np
from tqdm import tqdm
import pandas as pd


from parsers import (
    isp_workflow,
    check_redlining
)

In [2]:
# inputs
fn_acs = '../data/census/aggregated_tables_plus_features.csv.gz'
pattern_hughes = '../data/intermediary/offers/hughes/*/*.geojson.gz' # pattern for all data collected from lookup tools
pattern_xfinity = '../data/intermediary/offers/xfinity/*/*.geojson.gz'
pattern_viasat =  "../data/intermediary/offers/viasat/*/*.geojson.gz"

# outputs
fn_hughes = "../data/output/speed_price_hughes.csv.gz"
fn_xfinity = '../data/output/speed_price_xfinity.csv.gz'
fn_viasat = '../data/output/speed_price_viasat.csv.gz'

# params
n_jobs = 20
recalculate = True

In [3]:
# This is from Census data we crunched in the previous notebook.
acs = pd.read_csv(fn_acs, dtype={'geoid': str, 'block_group': str})

# These are the columns we're going to bring to merge with lookup responses.
acs_cols = [
    'geoid', 'race_perc_non_white','income_lmi', 
    'ppl_per_sq_mile', 'n_providers', 'income_dollars_below_median',
    'internet_perc_broadband', 'median_household_income'
]

Count Number of Offers Scraped

In [7]:
def count_addresses(fn):
    """
    How many addresses did we successfully collect in each file?
    """
    import gzip
    count = 0
    with gzip.open(fn, 'rb') as f:
        for _ in f.readlines():
            count += 1
    return count 

def count_successful_addresses(pattern, n_jobs=20):
    """
    For all files in `pattern`, sees how many addresses were successfully counted.
    Uses multiprocessing to speed things up.
    """
    files = glob.glob(pattern)
    count = 0
    with multiprocess.get_context("spawn").Pool(n_jobs) as pool:
        for _count in tqdm(pool.imap_unordered(count_addresses, files), 
                           total=len(files)):
            count += _count
    return count

In [8]:
# print number of offers collected for each ISP
hughes_count = count_successful_addresses(pattern_hughes, n_jobs=n_jobs)
xfinity_count = count_successful_addresses(pattern_xfinity, n_jobs=n_jobs)
viasat_count = count_successful_addresses(pattern_viasat, n_jobs=n_jobs)
all_records = hughes_count + xfinity_count + viasat_count

print(f"""Hughes Net: {hughes_count}
Xfinity: {xfinity_count}
ViaSat: {viasat_count}
Total: {all_records}""")

100%|██████████| 95/95 [00:00<00:00, 191.82it/s]

Hughes Net: 700
Xfinity: 276
ViaSat: 95
Total: 1071


## Hughes Net

In [9]:
if not os.path.exists(fn_hughes) or recalculate:
    # find the data we collected for each block group.
    data_hughes = []
    files = glob.glob(pattern_hughes)
    with multiprocess.Pool(n_jobs) as pool:
        # create parallel jobs that parse each block group of data using `hughes_workflow`.
        for record in tqdm(pool.imap_unordered(isp_workflow, files), 
                           total=len(files)):
            data_hughes.extend(record)
    hughes = pd.DataFrame(data_hughes)
    del data_hughes
    
    # check HOLC-grades for each address, and the distance to download speeds at or above 200 Mbps
    hughes = check_redlining(hughes)
    # merge census data, and save the file
    hughes_acs = hughes.merge(acs[acs_cols], how='left',
                        left_on='geoid', right_on='geoid')
    hughes_acs = hughes_acs[[c for c in hughes_acs.columns if c != 'geoid']]
    hughes_acs.to_csv(fn_hughes, index=False, compression='gzip')
else:
    hughes_acs = pd.read_csv(fn_hughes)

100%|██████████| 1/1 [00:00<00:00,  1.71it/s]


In [10]:
# start and end collection datetime
[datetime.fromtimestamp(hughes_acs.collection_datetime.min()), 
 datetime.fromtimestamp(hughes_acs.collection_datetime.max())]

[datetime.datetime(2023, 12, 5, 19, 15, 55, 406230),
 datetime.datetime(2023, 12, 5, 21, 56, 55, 37173)]

In [11]:
len(hughes_acs)

700

In [12]:
hughes_acs.redlining_grade.value_counts(normalize=True)

redlining_grade
C    0.714719
D    0.186646
B    0.097117
A    0.001517
Name: proportion, dtype: float64

Xfinity

In [13]:
if not os.path.exists(fn_xfinity) or recalculate:
    # find the data we collected for each block group.
    data_xfinity = []
    files = glob.glob(pattern_xfinity)
    with multiprocess.Pool(n_jobs) as pool:
        # create parallel jobs that parse each block group of data using `isp_workflow`.
        for record in tqdm(pool.imap_unordered(isp_workflow, files), 
                           total=len(files)):
            data_xfinity.extend(record)
    xfinity = pd.DataFrame(data_xfinity)
    del data_xfinity
        
    # check HOLC-grades for each address, and the distance to download speeds at or above 200 Mbps
    xfinity = check_redlining(xfinity)
    # merge census data, and save the file
    xfinity_acs = xfinity.merge(acs[acs_cols], how='left',
                        left_on='geoid', right_on='geoid')
    xfinity_acs = xfinity_acs[[c for c in xfinity_acs.columns if c != 'geoid']]
    xfinity_acs.to_csv(fn_xfinity, index=False, compression='gzip')
else:
    xfinity_acs = pd.read_csv(fn_xfinity)

100%|██████████| 1/1 [00:00<00:00,  7.09it/s]


In [14]:
# start and end collection datetime
[datetime.fromtimestamp(xfinity_acs.collection_datetime.min()), 
 datetime.fromtimestamp(xfinity_acs.collection_datetime.max())]

[datetime.datetime(2023, 12, 5, 19, 16, 14, 337115),
 datetime.datetime(2023, 12, 5, 21, 56, 50, 758039)]

In [15]:
len(xfinity_acs)

276

In [16]:
xfinity_acs.redlining_grade.value_counts(normalize=True)

redlining_grade
C    0.678431
D    0.203922
B    0.117647
Name: proportion, dtype: float64

Viasat

In [17]:
if not os.path.exists(fn_viasat) or recalculate:
    # find the data we collected for each block group.
    data_viasat = []
    files = glob.glob(pattern_viasat)
    with multiprocess.Pool(n_jobs) as pool:
        # create parallel jobs that parse each block group of data using `isp_workflow`.
        for record in tqdm(pool.imap_unordered(isp_workflow, files), 
                           total=len(files)):
            data_viasat.extend(record)
    viasat = pd.DataFrame(data_viasat)
    del data_viasat
        
    # check HOLC-grades for each address, and the distance to download speeds at or above 200 Mbps
    viasat = check_redlining(viasat)
    # merge census data, and save the file
    viasat_acs = viasat.merge(acs[acs_cols], how='left',
                        left_on='geoid', right_on='geoid')
    viasat_acs = viasat_acs[[c for c in viasat_acs.columns if c != 'geoid']]
    viasat_acs.to_csv(fn_viasat, index=False, compression='gzip')
else:
    viasat_acs = pd.read_csv(fn_viasat)

100%|██████████| 1/1 [00:00<00:00, 13.71it/s]


In [18]:
# start and end collection datetime
[datetime.fromtimestamp(viasat_acs.collection_datetime.min()), 
 datetime.fromtimestamp(viasat_acs.collection_datetime.max())]

[datetime.datetime(2023, 12, 5, 19, 16, 2, 321100),
 datetime.datetime(2023, 12, 5, 19, 38, 27, 988119)]

In [19]:
len(viasat_acs)

95

In [20]:
viasat_acs.redlining_grade.value_counts(normalize=True)

redlining_grade
C    0.609756
D    0.329268
B    0.060976
Name: proportion, dtype: float64